SQL operation usage in code

This report shows the SQL operation usages across the projects. It is a good way to understand the SQL usage patterns and identify call sites.

In [ ]:
import pandas as pd
import warnings
import moderne_pkg.helpers as helpers

warnings.simplefilter("ignore")

args = helpers.get_notebook_args_from_env()

# read data table file
file_name = args["filteredDataTableFileName"]
df = pd.read_csv(file_name, on_bad_lines='skip', skip_blank_lines=True)

# remove empty rows
df = df.dropna(how="all")

# rename columns
df = df.rename(columns={ "repositoryOrigin": "Origin", "repositoryPath": "Path", "repositoryBranch": "Branch", "sourcePath": "File", "table": "Table", "column": "Column", "operation": "Operation" })

# Add new columns
df["Repository"] = ""
df["Organization"] = ""

# attempt to split path into two columns
try:
    df[["Organization", "Repository"]] = df["Path"].str.split("/", n=1, expand=True)
except:
    pass

df.drop(columns=["Path"], inplace=True)

df = df[["Table","Column","Operation","Repository","File", "Origin", "Organization", "Branch"]]

helpers.moderne_data_grid(df)